# **Classification**

Classification is used to separate data points into classes of different labels. In this example we will be using Tensorflow estimtor to classify flowers based on the Iris Dataset

This is based on the Tensorflow website https://www.tensorflow.org/tutorials/estimator/premade


# **Imports and Setup**

In [1]:
%tensorflow_version 2.x # this line is not required unless you are in a notebook

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

Dataset 

This specific dataset separates flowers into 3 different classes of species
*   Setosa
*   Versicolor
*   Virginica

The information about the flowers are:
*   sepal length
*   petal length
*   sepal width
*   petal width

In [3]:
# Define some constants
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [4]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [5]:
train = pd.read_csv(train_path, names = CSV_COLUMN_NAMES, header = 0)
test = pd.read_csv(test_path, names = CSV_COLUMN_NAMES, header = 0)

In [6]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [7]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [8]:
#train_y.head()
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [9]:
train.shape

(120, 4)

Input Function

We now define a model using a Tensorflow Estimator. An Estimator is any class derived from the tf.estimator.Estimator. 

To write a Tensorflow program based on  pre-made Estimators, perform the following tasks:


*   Create one or more input functions
*   Define the model's feature columns   
*   Instantiate an Estimator, specifying the feature columns and various hyperparameters.
*   Call one or more methods on the Estimator object, passing the appropriate input function as the source of the data.

An input function is a function that returns a tf.data.Dataset object which outputs the following two-element tuple:

features - A Python dictionary in which:

*   Each key is the name of a feature
*   Each value is an array containing all of that feature's values.

label - An array containing the values of the label for every example








In [10]:
# #Just to demonstrate the format of the input function, here's a simple implementation
# def input_evaluation_set():
#     features = {'SepalLength': np.array([6.4, 5.0]),
#                 'SepalWidth':  np.array([2.8, 2.3]),
#                 'PetalLength': np.array([5.6, 3.3]),
#                 'PetalWidth':  np.array([2.2, 1.0])}
#     labels = np.array([2, 1])
#     return features, labels

Tensorflow's Dataset API (https://www.tensorflow.org/guide/data) can handle a lot of common cases for you. For example, using the Dataset API, you can easily read in records from a large collection of files in parallel and join them into a single stream. 

To keep things simple we are going to load the data with pandas, and built an input pipeline

In [11]:
def input_fn(features, labels, training = True, batch_size = 256):
  #Converts the inputs to a Dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  #Shuffle and repeat if you are in training mode
  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)

**Feature Columns**


In [12]:
my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


**Building the Model**

And now we are ready to choose a model. For classification tasks there are variety of different estimators/models that we can pick from.
Some options are:
*   DNNClassifier(Deep Neural Network)
*   LinearClassifier

We would be using DNNClassifier here




In [13]:
classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns,
    #Two hidden layers of 30 and 10 nodes respectively
    hidden_units=[30,10],
    #The model must choose between 3 classes
    n_classes=3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpn3jiwd6e', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


**Training**

The steps argument here tells the classifier to run for 5000 steps. Modifying this might change the result. Keep in mind that more is not always better.

In [14]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    #lambda is a function and whatever is after the colon is what the lambda function does. What is great about this is that lambda helps to define or call a function in one line
    steps=5000
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO

**Evaluation**

In [15]:
eval_result = classifier.evaluate(input_fn = lambda:input_fn(test, test_y, training = False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-13T15:20:45Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpn3jiwd6e/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.29054s
INFO:tensorflow:Finished evaluation at 2020-07-13-15:20:46
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.8333333, average

In [16]:
print(eval_result)

{'accuracy': 0.8333333, 'average_loss': 0.48265857, 'loss': 0.48265857, 'global_step': 5000}


**Prediction**

In [17]:
def input_fn(features, batch_size=256):
  # Convert the inputs to a Dataset without label
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)


features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted")
for feature in features:
  valid = True
  while valid:
    val = input(feature + ":")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)] 

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
  print(pred_dict)
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100 * probability))





Please type numeric values as prompted
SepalLength:2.5
SepalWidth:2.6
PetalLength:4.3
PetalWidth:6.2
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpn3jiwd6e/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-4.9997745, -0.7402899,  0.7534181], dtype=float32), 'probabilities': array([0.00258419, 0.18289196, 0.8145238 ], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Virginica" (81.5%)


In [18]:
# Here is some example input and expected classes you can try above
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength' : [5.1, 5.9, 6.9],
    'SepalWidth' : [3.3, 3.0, 3.1],
    'PetalLength' : [1.7, 4.2, 5.4],
    'PetalWidth' : [0.5, 1.5, 2.1],
}

**Clustering**

Clustering is a Machine Learning technique that involves the grouping of data points. In theory, data points that are in the same group should have similar properties and/or features, while data points in different groups should have highly dissimilar properties and/or features.

Unfortunately, there are issues with teh current version of Tensorflow and the implementation for KMeans

Basic Algorithm for K-Means. 

*   Step 1: Randomly pick K points to place K centroids
*   Step 2: Assign all of the data points to the centroids by distance. The closest centroid to a point is the one it is assigned to
*   Step 3: Average of all of the points belonging t0 find the middle of those clusters (Center of mass). Place the corresponding centroids into that position.
*   Step 4: Reaasign every point once again to the closest centroid
*   Step 5: Repeat steps 3-4 until no point changes which centroid it belogs to.


**Hidden Markov Models**

A hidden markov model is a finite set of states, each of which is assoiciated with a (generally multidimensional) probability distribution. Transitions among the states are governed by a set of probabilities called transition probabilities. 
(http://jedlik.phy.bme.hu/~gerjanos/HMM/node4.html)

A hidden markov model works with probabilities to predict future events or states. In this section, we will learn how to create a hidden markov model that can predict the weather.

This section is based on the TensorFlow tutorial.
https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/HiddenMarkovModel




**Data**

For a markov model we are only interested in probability distributions that have to do with states. We can find these probabilities with large datasets or may already have these values. 

States: In each markov model we have a finite set of states. These states could be something like warm and cold or red, blue, green and blue. These states are hidden within the model, which means we do not directly observe them.

Observation: Each state has a particular outcome or observation associated with it based on a probability distribution . An example of this is the following:On a hot day Tim has a 80% chance of being happy and a 20% chance of being sad.

Transitions: Each state will have a probability of transitioning to a different state. An Example is the following: A cold day has a 30% chance of being followed by a hot day and a 70% chance of being followed by another cold day.

To create a hidden markov model we need.

*   States
*   Observation Distribution
*   Transition Distribution


**Imports and Setup**

In [19]:
import tensorflow_probability as tfp 
import tensorflow as tf

Weather model

Taken from Tensorflow documentation
https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/HiddenMarkovModel

We will model a simple weather system and try to predict the temperature on each day given the following information:

1.   Cold days are encoded by a 0 and hot days are encoded as 1
2.   The first day in our sequence has an 80% chance of being cold
3.   A cold day has a 30 % chance of being followed by a hot day
4.   A hot day has a 20% chance of being followed by a cold day
5.   On each day the teamperature is normally distributed with mean and  deviation 0 and 5 on a cold day and mean and devition of 15 and 10 on a hot day.

In [20]:
tfd = tfp.distributions  # making a shortcut for later on
initial_distribution = tfd.Categorical(probs = [0.8, 0.2]) # Refer to point 2 above
transition_distribution = tfd.Categorical(probs = [[0.7, 0.3], [0.2, 0.8]]) # Based on point 3 and 4 above
observation_distribution = tfd.Normal(loc=[0.,15.], scale=[5.,10.]) # Refer to point 5 above
# the loc argument represents the mean and the scale represents the deviation

We've now created distribution variables to model our system and it's time to create the hidden markov model

In [21]:
model = tfd.HiddenMarkovModel(
    initial_distribution=initial_distribution,
    transition_distribution = transition_distribution,
    observation_distribution = observation_distribution,
    num_steps=7)

The number of steps represents the number of days that we would like to predict information for. In this case, we have chosen entire week

Now to get the expected temperatures on each day we can do the following

In [22]:
mean = model.mean()

# due to the way TensorFlow works on a lower level we need to evaluate part of the graph
# frokm within a session to see the value of this tensor

# in the newer version of TensorFlow we need to use tf.compat.v1.Session() rather than just tf.Session()
with tf.compat.v1.Session() as sess:
  print(mean.numpy())

[2.9999998 5.9999995 7.4999995 8.25      8.625001  8.812501  8.90625  ]


If we run this model for 'n' number of times, we would get similar values.

**Re-creating with different values and probabilities**

In [23]:
tfd = tfp.distributions  # making a shortcut for later on
initial_distribution = tfd.Categorical(probs = [0.5, 0.5]) # Refer to point 2 above
transition_distribution = tfd.Categorical(probs = [[0.7, 0.3], [0.2, 0.8]]) # Based on point 3 and 4 above
observation_distribution = tfd.Normal(loc=[0.,15.], scale=[5.,10.]) # Refer to point 5 above
# the loc argument represents the mean and the scale represents the deviation

In [24]:
model = tfd.HiddenMarkovModel(
    initial_distribution=initial_distribution,
    transition_distribution = transition_distribution,
    observation_distribution = observation_distribution,
    num_steps=7)

In [25]:
mean = model.mean()

# due to the way TensorFlow works on a lower level we need to evaluate part of the graph
# frokm within a session to see the value of this tensor

# in the newer version of TensorFlow we need to use tf.compat.v1.Session() rather than just tf.Session()
with tf.compat.v1.Session() as sess:
  print(mean.numpy())

[7.5       8.25      8.625001  8.812501  8.90625   8.953125  8.9765625]


Conclusion

So that's it for the core learning algorithms in TensorFlow. Hopefully, you've learned about a few interesting tools that are pretty easy to use and practice. I'd encourage you to try out some of these algorithms on different datasets.